In [ ]:
import pandas as pd
import numpy as np
import importlib
import tTsTGrpUtils as tsutil
import utils_plots as uplots

In [ ]:
# load in all dls up to now
importlib.reload(tsutil)
root = "/host/verges/tank/data/daniel/3T7T/z/outputs/"
pth_04a_maps = "04a_dl_maps_08Oct2025-104443.pkl"
pth_04b_parc = "04b_dl_maps_parcel_08Oct2025-110220.pkl"
pth_05a_winComp = "05a_winStudy_08Oct2025-110338.pkl"
pth_05b_winComp_ic = "05b_stats_winStudy_grp_08Oct2025-110654.pkl"
pth_05c_winD = "05c_stats_winD_08Oct2025-140529.pkl"
pth_05d_btwD = "05d_btwD_08Oct2025-153038.pkl"

dl_04a_maps = tsutil.loadPickle(root + pth_04a_maps, verbose = True)
dl_04b_parc = tsutil.loadPickle(root + pth_04b_parc, verbose = True)
dl_05a_winComp = tsutil.loadPickle(root + pth_05a_winComp, verbose = True)
dl_05b_winComp_ic = tsutil.loadPickle(root + pth_05b_winComp_ic, verbose = True)
dl_05c_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)
dl_05d_btwD = tsutil.loadPickle(root + pth_05d_btwD, verbose = True)

In [ ]:
index = 12
dl_04a_slice = dl_04a_maps[index]
txt = tsutil.printItemMetadata(dl_04a_slice, return_txt=True)
print(f"dl_maps: {txt}")

# find corresponding index in other dls

dl_04b_slice = dl_04b_parc[index]
txt = tsutil.printItemMetadata(dl_04b_slice, return_txt=True)
print(f"dl_parc: {txt}")

dl_05a_slice = dl_05a_winComp[index]
txt = tsutil.printItemMetadata(dl_05a_slice, return_txt=True)
print(f"dl_winComp: {txt}")

dl_05b_slice = dl_05b_winComp_ic[index]
txt = tsutil.printItemMetadata(dl_05b_slice, return_txt=True)
print(f"dl_winComp_ic: {txt}")

dl_05c_slice = dl_05c_winD[index]
txt = tsutil.printItemMetadata(dl_05c_slice, return_txt=True)
print(f"dl_winD: {txt}")

dl_05d_slice = dl_05d_btwD[index]
txt = tsutil.printItemMetadata(dl_05d_slice, return_txt=True)
print(f"dl_btwD: {txt}")

In [ ]:
# print keys for each test item
print(f"dl_maps keys:\n\t{list(dl_04a_slice.keys())}")
print(f"dl_parc keys:\n\t{list(dl_04b_slice.keys())}")
print(f"dl_winComp keys:\n\t{list(dl_05a_slice.keys())}")
print(f"dl_grp_ic keys:\n\t{list(dl_05b_slice.keys())}")

In [ ]:
if dl_04a_slice['region'] == 'cortex':
    dl_maps_key = 'df_maps'
    dl_parc_key = 'df_maps_parc_glsr_mdn'
    dl_winComp_key = 'df_maps_parc_glsr_mdn_z'
    dl_grp_ic_key = 'df_maps_parc_glsr_mdn_z_TLE_ic'
else:
    dl_maps_key = 'df_maps'
    dl_parc_key = 'df_maps_parc_dk25_mdn'
    dl_winComp_key = 'df_maps_parc_dk25_mdn_z'
    dl_grp_ic_key = 'df_maps_parc_dk25_mdn_z_TLE_ic'

# print indices and col names for each
df_maps = dl_04a_slice[dl_maps_key]
if type(df_maps) is str:
    df_maps = tsutil.loadPickle(df_maps, verbose = True)
print(f"dl_maps <{df_maps.shape}>\n\t{list(df_maps.columns)}")

df_parc = dl_04b_slice[dl_parc_key]
if type(df_parc) is str:
    df_parc = tsutil.loadPickle(df_parc, verbose = True)
print(f"dl_parc <{df_parc.shape}>\n\t{list(df_parc.columns)}")

df_winComp = dl_05a_slice[dl_winComp_key]
if type(df_winComp) is str:
    df_winComp = tsutil.loadPickle(df_winComp, verbose = True)
print(f"dl_winComp <{df_winComp.shape}>\n\t{list(df_winComp.columns)}")

df_grp_ic = dl_05b_slice[dl_grp_ic_key]
if type(df_grp_ic) is str:
    df_grp_ic = tsutil.loadPickle(df_grp_ic, verbose = True)
print(f"dl_grp_ic <{df_grp_ic.shape}>\n\t{list(df_grp_ic.columns)}")

In [ ]:
specs  = { # all spec values to be in lists to allow for iteration across these values
    # directories
    'prjDir_root' : "/host/verges/tank/data/daniel/3T7T/z", 
    'prjDir_outs' : "/outputs",
    'prjDir_out_stats': "/outputs/stats",
    'prjDir_out_figs': "/outputs/figures",
    'prjDir_maps' : "/maps", # output directory for smoothed cortical maps
    'prjDir_dictLists': "/maps/dictLists",
    'prjDir_mapPths' : "/output/paths",
    'prjDir_maps_dfs': "/outputs/dfs/04a_maps_dfs",
    'prjDir_parc_dfs': "/outputs/dfs/04b_maps_parc",
    'prjDir_winComp_dfs': "/outputs/dfs/05a_winComp",
    'prjDir_grpFlip_dfs': "/outputs/dfs/05b_grpFlip",
    'prjDir_winD_dfs': "/outputs/dfs/05c_winD",
    'prjDir_btwD_dfs': "/outputs/dfs/05d_btwComp",

    'ctx': True, # whether to include cortical analyses
    'surf_ctx': ['fsLR-32k', 'fsLR-5k'],
    'parcellate_ctx': 'glasser', # parcellation to use, or None if no parcellation.
    'parc_lbl_ctx': 'glasser_int', # what name to fetch for parcellation values
    'lbl_ctx': ['midthickness', 'pial', 'white'], # pial, midthick, white, etc
    'ft_ctx': ['thickness', 'T1map'], # features: T1map, flair, thickness, FA, ADC
    'smth_ctx': [5, 10], # in mm
    
    'hipp': True, # whether to include hippocampal analyses
    'surf_hipp': ['den-0p5mm'],
    'parcellate_hipp': 'DK25',
    'parc_lbl_hipp': 'idx',
    'lbl_hipp': ['midthickness', "inner", "outer"], # outer, inner, midthickness, etc
    'ft_hipp': ['thickness', 'T1map'], # features: T1map, flair, thickness, FA, ADC
    'smth_hipp': [2, 5], # in mm
    
    # downsampling
    'ds_study': ['PNI'], # list of study codes to apply downsampling to
    'ds_foi': ['T1map'], # features to downsample
    'ds_res': [0.8], # resolution (in mm) to downsample volumes to. NOTE. should be same length as ds_foi with each value corresponding to that in ds_foi 
    'ds_vol_dir': '/downsampled_vols', # name of directory within project dir root
    
    # within study comparisons
    'col_grp': 'grp_detailed',  # column in df_demo with group labels
    'winComp_stats': ['z'], # what stats to run for within study comparisons ('z' for z-scoring, 'w' for w-scoring)
    #'covars': [demographics['age'], demographics['sex']],

    'ipsiTo' : 'L', # what hemisphere for controls ipsi should be mapped to
    'newQC': True
}


specs['prjDir_root'] + specs['ds_vol_dir']

In [ ]:
importlib.reload(tsutil)
root = "/host/verges/tank/data/daniel/3T7T/z/outputs/"
dl_pth = "05c_stats_winD_08Oct2025-140529.pkl"
dl = tsutil.loadPickle(root + dl_pth, verbose = True)

In [ ]:
tsutil.print_dict(dl, idx = 2)

In [ ]:
df_key = 'df_d_ic'
idx = 2
df = dl[idx][df_key]
df = tsutil.loadPickle(df)
print(f"df <{df.shape}>\n\t{list(df.columns)}")

In [ ]:
specs  = { # all spec values to be in lists to allow for iteration across these values
    # directories
    'prjDir_root' : "/host/verges/tank/data/daniel/3T7T/z", 
    'prjDir_outs' : "/outputs",
    'prjDir_out_stats': "/outputs/stats",
    'prjDir_out_figs': "/outputs/figures",
    'prjDir_maps' : "/maps", # output directory for smoothed cortical maps
    'prjDir_dictLists': "/maps/dictLists",
    'prjDir_mapPths' : "/output/paths",
    'prjDir_maps_dfs': "/outputs/dfs/04a_maps_dfs",
    'prjDir_parc_dfs': "/outputs/dfs/04b_maps_parc",
    'prjDir_winComp_dfs': "/outputs/dfs/05a_winComp",
    'prjDir_grpFlip_dfs': "/outputs/dfs/05b_grpFlip",
    'prjDir_winD_dfs': "/outputs/dfs/05c_winD",
    'prjDir_btwD_dfs': "/outputs/dfs/05d_btwComp",

    # downsampling
    'ds_study': ['PNI'], # list of study codes to apply downsampling to
    'ds_foi': ['T1map'], # features to downsample
    'ds_res': [0.8], # resolution (in mm) to downsample volumes to. NOTE. should be same length as ds_foi with each value corresponding to that in ds_foi 
    'ds_vol_dir': '/downsampled_vols', # name of directory within project dir root

    # analysis regions
    'ctx': True, # whether to include cortical analyses
    'surf_ctx': ['fsLR-32k', 'fsLR-5k'],
    'parcellate_ctx': 'glasser', # parcellation to use, or None if no parcellation.
    'parc_lbl_ctx': 'glasser_int', # what name to fetch for parcellation values
    'lbl_ctx': ['midthickness', 'pial', 'white'], # pial, midthick, white, etc
    'ft_ctx': ['thickness', 'T1map'], # features: T1map, flair, thickness, FA, ADC
    'smth_ctx': [5, 10], # in mm
    
    'hipp': True, # whether to include hippocampal analyses
    'surf_hipp': ['den-0p5mm'],
    'parcellate_hipp': 'DK25',
    'parc_lbl_hipp': 'idx',
    'lbl_hipp': ['midthickness', "inner", "outer"], # outer, inner, midthickness, etc
    'ft_hipp': ['thickness', 'T1map'], # features: T1map, flair, thickness, FA, ADC
    'smth_hipp': [2, 5], # in mm
        
    # within study comparisons
    'col_grp': 'grp_detailed',  # column in df_demo with group labels
    'winComp_stats': ['z'], # what stats to run for within study comparisons ('z' for z-scoring, 'w' for w-scoring)
    #'covars': [demographics['age'], demographics['sex']],

    'ipsiTo' : 'L', # what hemisphere for controls ipsi should be mapped to
    'newQC': True
}


In [ ]:
for reg_name, region, surfs, lbls, smths in zip( ['ctx', 'hipp'], [specs['ctx'], specs['hipp']], [specs['surf_ctx'], specs['surf_hipp']], [specs['lbl_ctx'], specs['lbl_hipp']], [specs['smth_ctx'], specs['smth_hipp']] ) :
    print(f"[get_dsMaps_pths_iter] [{region}] Processing {reg_name} ...")
    if not region: # if false, skip
        continue
    for surf in surfs:
        for lbl in lbls:
            for smth in smths:
                print(f"\tProcessing surf `{surf}`, lbl `{lbl}`, smth `{smth}` ...")

In [ ]:
dl_parc = tsutil.loadPickle("/host/verges/tank/data/daniel/3T7T/z/outputs/04d_dl_maps_18Oct2025-154136.pkl")
tsutil.print_dict(dl_parc, df_print=False)

In [ ]:

importlib.reload(tsutil)
importlib.reload(uplots)
import pandas as pd

dl_parc = tsutil.loadPickle("/host/verges/tank/data/daniel/3T7T/z/outputs/04e_dl_maps_parcel_18Oct2025-221826.pkl")
for i, item in enumerate(dl_parc):
    difdsRes = False
    res = item.get('downsampledRes', None)
    
    if res == str(0.8):
        difdsRes = True
        #tsutil.printItemMetadata(item, idx = i, return_txt=False)
    else: # do not match on get pair
        difdsRes = False
        
    tsutil.printItemMetadata(item, idx = i, return_txt=False)
    matches = tsutil.get_pair(dl = dl_parc, idx = i, 
                              mtch = ['region', 'surf', 'label', 'feature', 'smth', 'downsampledRes'], 
                              difdsRes = difdsRes, skip_idx = [i])
    print(matches)
    continue
    if isinstance(matches, list):
        print(f"\t{res}: {len(matches)} - {matches}")
        idx_other = matches[0] # take first match
    elif isinstance(matches, int):
        idx_other = matches
    
    item_other = dl_parc[matches]
    



In [ ]:
importlib.reload(tsutil)
importlib.reload(uplots)
key_ft = 'feature' # key for feature in dict
foi = ['T1map'] # features of interest
key_ids = [['ctrl_IDs'],['TLE_R_IDs', 'TLE_L_IDs']]
pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/05b_stats_winStudy_grp_18Oct2025-225832.pkl"
dl_05b_winComp_ic = tsutil.loadPickle(pth, verbose = True)
dl_05b_winComp_ic = tsutil.filt_dl(dl = dl_05b_winComp_ic, 
                                   key_ft = key_ft, foi = foi, 
                                   key_ids = key_ids)

# NOTE. pass z-scores only. Need changes for w-scores
uplots.raw2Z_vis(dl = dl_05b_winComp_ic, save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/rawToZ", 
               key_dfs_raw = ['df_maps_parc-glsr_mean', 'df_maps_parc-dk25_mean'],
               marks = True, ctrl_only = True, test = False)


In [ ]:
root = "/host/verges/tank/data/daniel/3T7T/z/outputs/"
pth_04a_raw = "04a_dl_maps_08Oct2025-104443.pkl"
pth_04b_parc = "04b_dl_maps_parcel_08Oct2025-110220.pkl"
pth_05a_winComp = "05a_winStudy_08Oct2025-110338.pkl" # z or w scores
pth_05b_winComp_ic = "05b_stats_winStudy_grp_08Oct2025-110654.pkl" # flipped
pth_05c_winD = "05c_stats_winD_08Oct2025-140529.pkl" # Has all data from above maps as well. D score btw px and ctrl z-scores
pth_05d_btwD = "05d_btwD_08Oct2025-153038.pkl" # difference in D scores between studies
if 'dl_winD' not in globals():
    dl_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)


In [ ]:
tsutil.print_dict(dl_winD)

In [ ]:

key_ft = 'feature' # key for feature in dict
foi = ['thickness', 'T1map'] # features of interest
key_ids = [['ctrl_IDs'],['TLE_R_IDs', 'TLE_L_IDs']]

key_df = 'df_d_ic'
idx_d = 'd_TLE_ic_ipsiTo-L'
row_a = tsutil.loadPickle(dl_winD[5]['df_d_ic']).loc[idx_d]
row_b = tsutil.loadPickle(dl_winD[10]['df_d_ic']).loc[idx_d]
uplots.correl(row_a, row_b, method='Pearson')
